In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import numpy as np
from scipy.io import wavfile
import tensorflow as tf
import utilities as util

import matplotlib.pylab as plt 

import warnings
warnings.filterwarnings("ignore", category=wavfile.WavFileWarning)

In [4]:
%%time
nps_uniq, dfreqs, notes = util.precompute_spect()
smallest_nps = 44100/24 * 4/3 
smallest_nps = 2**int(np.log2(smallest_nps))
note_freq = util.frequencies('A0', 'D8')
sample_files, target_vals = util.gather_samples(util.note_names('C1', 'G1'), 2) # Guitar, Trombone, Piano
init = tf.global_variables_initializer()

time_steps = 88
n_inputs=time_steps*len(notes)

CPU times: user 2.38 ms, sys: 0 ns, total: 2.38 ms
Wall time: 2.05 ms


In [3]:
%%time

all_specs = []
for sf in sample_files:
    s = util.preprocess(sf, nps_uniq, dfreqs, notes, smallest_nps, note_freq)
    all_specs.append(s)

CPU times: user 15min 44s, sys: 1.83 s, total: 15min 46s
Wall time: 15min 49s


In [5]:
X = tf.placeholder(tf.float16, shape=(None,n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

In [6]:
%%time

n_hidden1 = 2500
n_hidden2 = 500
n_outputs = 3

with tf.name_scope("network"):
    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1", activation=tf.nn.relu) # Play with activations
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu) # Play with activations
    net_results = tf.layers.dense(hidden2, n_outputs, name="outputs")

CPU times: user 29.3 ms, sys: 1.83 ms, total: 31.1 ms
Wall time: 30.9 ms


In [7]:
%%time

with tf.name_scope("loss_sparse"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=net_results)
    loss = tf.reduce_mean(xentropy, name="loss_sparse")

CPU times: user 3.54 ms, sys: 0 ns, total: 3.54 ms
Wall time: 3.33 ms


In [8]:
%%time

with tf.name_scope("loss_soft"):
    xentropy = tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=net_results)
    loss = tf.reduce_mean(xentropy, name="loss_soft")

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

CPU times: user 56.7 ms, sys: 2.99 ms, total: 59.7 ms
Wall time: 59.3 ms


In [ ]:
%%time

with tf.name_scope("loss_sparse"):
    xentropy = tf.nn.(labels=y, logits=net_results)
    loss = tf.reduce_mean(xentropy, name="loss_sparse")

In [ ]:
%%time
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [ ]:
%%time
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(tf.cast(net_results, tf.float32), y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [ ]:
%%time

spectrograms = []
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    spectrograms = sess.run(all_specs)

In [ ]:
%%time
new_specs22 = []
for i in range(len(all_specs)):
    new_spectrograms = []
    end = len(np.trim_zeros(spectrograms[i].sum(1), 'b'))
    new_spectrograms = np.concatenate(spectrograms[i][:,:end]).ravel()
#     print(new_spectrograms.shape)
    new_specs22.append(new_spectrograms)

In [ ]:
%%time
saver = tf.train.Saver()

n_epochs = 10
batch_size = 3
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(n_epochs):
#         for iteration in range(len(new_specs22) // batch_size):
        X_batch = new_specs22
        Y_batch = target_vals
        sess.run(training_op, feed_dict={X: np.stack(X_batch),
                                         y: np.array(Y_batch)})
#         acc_train = accuracy.eval(feed_dict={X: X_batch, y: [0,1,2]})
        acc_val = accuracy.eval(feed_dict={X: X_batch, y: Y_batch})
        print(epoch, "Val accuracy:", acc_val)

    save_path = saver.save(sess, "./FirstNetwork.ckpt")

In [ ]:
new_sample_files = []
new_target_vals = []
new_location = '../samples/'
new_instruments = ['Trombone', 'Guitar', 'Piano']
for i, new_name in enumerate(new_instruments):
    for new_note in (util.note_names('C1', 'F1')):
        for j in range(2, 3):
            new_file_name = new_location + new_name+"/"+new_name +"_"+new_note[:-1]+"("+str(j)+").wav"
            new_sample_files.append(new_file_name)
            new_target_vals.append(i)